In [13]:
pip install geoviews

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install hvplot

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install bokeh

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Dependencies and Setup
#import hvplot.pandas
#import pandas as pd
#import hvplot.pandas
#import holoviews as hv
#hv.extension('bokeh')
#from bokeh.plotting import show
#import geoviews


import hvplot.pandas
import pandas as pd


# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

import requests
import json

import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key



# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,302.14,74,100,6.17,CC,1731389527
1,1,vilyuchinsk,52.9306,158.4028,275.30,75,15,12.45,RU,1731389648
2,2,les cayes,18.2000,-73.7500,298.45,94,100,5.84,HT,1731389649
3,3,mount pearl,47.5166,-52.7813,281.11,93,100,11.83,CA,1731389531
4,4,waitangi,-43.9535,-176.5597,285.27,72,18,1.34,NZ,1731389652


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
%%capture --no-display

#NOTE: IF YOU CAN'T SE THE MAP TILE, PLEASE ALTERNATE BETWEEN TRUE OR FALSE VALUES FOR "GEO" PARAMETER

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    geo = False,
    tiles = "OSM",
    color = "City"
)

# Display the map


map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_df=city_data_df.loc[(city_data_df["Max Temp"]>=291)&(city_data_df["Wind Speed"]<4.5)&(city_data_df["Cloudiness"]==0)&(city_data_df["Max Temp"]<=301)]
#Temperature is expressed in Kelvins, so ranges were closely converted to 27 and 21 degrees celcius

# Drop any rows with null values
# There were no null 

# Display sample data
narrow_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,upington,-28.4478,21.2561,292.36,27,0,1.54,ZA,1731389688
29,29,alaghsas,17.0187,8.0168,291.09,34,0,2.57,NE,1731389689
33,33,tura,25.5198,90.2201,300.54,49,0,1.10,IN,1731389572
34,34,kangding,30.0513,101.9603,291.34,38,0,1.15,CN,1731389540
83,83,arauca,7.0847,-70.7591,298.13,88,0,2.03,CO,1731389768
145,145,iranshahr,27.2025,60.6848,298.34,22,0,2.06,IR,1731389855
149,149,manta,-0.9500,-80.7333,295.03,83,0,2.06,EC,1731389861
202,202,doffing,26.2745,-98.3858,296.88,87,0,2.06,US,1731389942
203,203,seminole,28.7086,-81.2081,295.60,88,0,0.00,US,1731389943
244,244,avare,-23.0986,-48.9258,293.02,60,0,2.31,BR,1731390000


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=narrow_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
28,upington,ZA,-28.4478,21.2561,27,
29,alaghsas,NE,17.0187,8.0168,34,
33,tura,IN,25.5198,90.2201,49,
34,kangding,CN,30.0513,101.9603,38,
83,arauca,CO,7.0847,-70.7591,88,
145,iranshahr,IR,27.2025,60.6848,22,
149,manta,EC,-0.9500,-80.7333,83,
202,doffing,US,26.2745,-98.3858,87,
203,seminole,US,28.7086,-81.2081,88,
244,avare,BR,-23.0986,-48.9258,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel

category="accommodation.hotel"

radius = 10000
params = {
   "apiKey":geoapify_key,
    "categories":category
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude=hotel_df.loc[index, "Lat"]
    longitude=hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
upington - nearest hotel: Die Eiland Holiday Ressort
alaghsas - nearest hotel: Hôtel Zagado
tura - nearest hotel: Hotel Polo Orchid
kangding - nearest hotel: 溜溜城格桑花大酒店
arauca - nearest hotel: Hotel Nova Park
iranshahr - nearest hotel: هتل قصِر
manta - nearest hotel: Hotel Santa Fe
doffing - nearest hotel: Executive Inn & Suites
seminole - nearest hotel: Best Western Plus Sanford
avare - nearest hotel: Hotel Estãncia de Avaré
awjilah - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
rio de janeiro - nearest hotel: Ibis Rio porto atlantico
conchal - nearest hotel: No hotel found
mariental - nearest hotel: Mariental Hotel
matoury - nearest hotel: La Marmotte
saint croix - nearest hotel: The Buccaneer
akjoujt - nearest hotel: Hotel Fatima فندق فاطمة
bam - nearest hotel: هتل جهانگردی بم
ojinaga - nearest hotel: Hotel
tchintabaraden - nearest hotel: No hotel found
moultrie - nearest hotel: Hampton


,City,Country,Lat,Lng,Humidity,Hotel Name
28,upington,ZA,-28.4478,21.2561,27,Die Eiland Holiday Ressort
29,alaghsas,NE,17.0187,8.0168,34,Hôtel Zagado
33,tura,IN,25.5198,90.2201,49,Hotel Polo Orchid
34,kangding,CN,30.0513,101.9603,38,溜溜城格桑花大酒店
83,arauca,CO,7.0847,-70.7591,88,Hotel Nova Park
145,iranshahr,IR,27.2025,60.6848,22,هتل قصِر
149,manta,EC,-0.9500,-80.7333,83,Hotel Santa Fe
202,doffing,US,26.2745,-98.3858,87,Executive Inn & Suites
203,seminole,US,28.7086,-81.2081,88,Best Western Plus Sanford
244,avare,BR,-23.0986,-48.9258,60,Hotel Estãncia de Avaré


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display


#NOTE: IF YOU CAN'T SE THE MAP TILE, PLEASE ALTERNATE BETWEEN TRUE OR FALSE VALUES FOR "GEO" PARAMETER

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    geo = False,
    tiles = "OSM",
    color = "City",
    hover_cols=["Hotel Name","Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (City,Humidity,Hotel Name,Country)